# Trablho de Conclusão de Curso - UNIVESP - 1S/2025- Notebook1

# Código: tcc530-sala-002grupo-002

## 1.0 Packges

### Packges Import

In [1]:
# Packages

# Bibliotecas Gerais
import numpy                   as np
import pandas                  as pd
import matplotlib.pyplot       as plt
import seaborn                 as sns
import sys
import time
from scipy                     import stats

#Bibliotecas de Regressão
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing   import StandardScaler
from sklearn.svm             import SVR
from sklearn.tree            import DecisionTreeRegressor
from sklearn.linear_model    import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, ElasticNetCV, HuberRegressor, LassoLars, BayesianRidge
from sklearn.linear_model    import SGDRegressor
from sklearn.ensemble        import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor

# Bibliotecas de Clusterização
import scipy.cluster.hierarchy as shc
from sklearn.cluster           import KMeans
from sklearn.metrics           import silhouette_score, davies_bouldin_score,v_measure_score
from sklearn.mixture           import GaussianMixture
from sklearn.neural_network    import MLPRegressor
from sklearn.metrics           import r2_score, mean_absolute_error,mean_squared_error
from sklearn.model_selection   import GridSearchCV
from sklearn.model_selection   import RandomizedSearchCV
from sklearn.neighbors         import NearestNeighbors
from sklearn.cluster           import DBSCAN
from sklearn.cluster           import AgglomerativeClustering

# Bibliotecas Classificação
from sklearn                       import metrics
from sklearn.preprocessing         import MinMaxScaler
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.linear_model          import LogisticRegression
from sklearn.tree                  import DecisionTreeClassifier
from sklearn.ensemble              import RandomForestClassifier
from sklearn.naive_bayes           import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.svm                   import SVC
from sklearn.neural_network        import MLPClassifier
from sklearn.metrics               import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics               import classification_report
from sklearn.metrics               import ConfusionMatrixDisplay
from sklearn.metrics               import RocCurveDisplay

# Bibliotecas de Recomendação
from sklearn.metrics.pairwise      import cosine_similarity

# Redução de Dimensionalidade
from sklearn.decomposition         import PCA
from sklearn.decomposition         import TruncatedSVD

### Packges Version Control

In [2]:
pip freeze --local > ../requeriments.txt

Note: you may need to restart the kernel to use updated packages.


## 2.0 Data Read

In [3]:
link   = '../dados/dados_filtrados_2014.csv'
df2014 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2015.csv'
df2015 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2016.csv'
df2016 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2017.csv'
df2017 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2018.csv'
df2018 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2019.csv'
df2019 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2020.csv'
df2020 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2021.csv'
df2021 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2022.csv'
df2022 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2023.csv'
df2023 = pd.read_csv(link,sep=",",engine='python')

## 3.0 Data Join

In [4]:
df2021 = df2021.rename(columns={"ANO_NASC": "DT_NASC"})
df2022 = df2022.rename(columns={"ANO_NASC": "DT_NASC"})
df2023 = df2023.rename(columns={"ANO_NASC": "DT_NASC"})

In [5]:
df_full = pd.concat([df2014, df2015,df2016,df2017,df2018,df2019,df2020,df2021,df2022,df2023], axis=0, ignore_index=True)

In [6]:
def verificacao_nulos(df_true,target_clean,col_ok):

  df = df_true.copy()

  target_clean_por = 100*target_clean

  names_df  = df.columns
  nnames_df = len(names_df)
  porcent_df_null = df.isnull().sum()/df.shape[0]

  for i in range(0,nnames_df):
    
      if(porcent_df_null[i]>target_clean):

          print('A feature %s possui mais do que %.2f porcento de dados nulos'%(names_df[i],target_clean_por))

      else:

          col_ok.append(names_df[i])
          

In [ ]:
target_clean = 0.5
col_ok       = []
verificacao_nulos(df_full,target_clean,col_ok)

In [ ]:
df_clean = df_full[col_ok].dropna(axis=0).reset_index(drop=True)
df_clean.head(5)

In [ ]:
reduct = 100*(1-(df_clean.shape[0]/df_full.shape[0]))
print('Full Size: %d'%df_full.shape[0])
print('Clean Size: %d'%df_clean.shape[0])
print('Reduction in Size: %f'%reduct)

In [ ]:
df_clean.columns

In [ ]:
df_clean = df_clean.ffill()

In [ ]:
target_clean = 0.5
col_ok       = []
verificacao_nulos(df_clean,target_clean,col_ok)

In [ ]:
link   = '../dados/municipios_sp_edited.csv'
df_muni_sp = pd.read_csv(link,sep=",")
df_muni_sp = df_muni_sp.drop(['ID_ESTADO','ESTADO'],axis=1)
df_muni_sp.head()

In [ ]:
df_muni_sp = df_muni_sp.rename(columns={"ID_MUNICIPIO": "ID_MUNICIP"})

In [ ]:
df_clean = df_clean.merge(df_muni_sp,on=['ID_MUNICIP'])
df_clean.head(5)

In [ ]:
df_clean.dtypes

In [ ]:
df_clean['DT_NASC_YEAR'] = pd.DatetimeIndex(df_clean['DT_NASC']).year
df_clean = df_clean.drop(['DT_NASC'],axis=1)

df_clean['DT_NOTIFIC']   = pd.to_datetime(df_clean['DT_NOTIFIC'])
df_clean['DT_SIN_PRI']   = pd.to_datetime(df_clean['DT_SIN_PRI'])
df_clean['DT_NASC_YEAR'] = pd.to_datetime(df_clean['DT_NASC_YEAR'])
df_clean['DT_INVEST']    = pd.to_datetime(df_clean['DT_INVEST'])
df_clean['DT_ENCERRA']   = pd.to_datetime(df_clean['DT_ENCERRA'])

df_clean['REGIAO']     = df_clean['REGIAO'].astype('category')
df_clean['MUNICIPIO']  = df_clean['MUNICIPIO'].astype('category')
df_clean['ID_AGRAVO']  = df_clean['ID_AGRAVO'].astype('category')
df_clean['ID_MUNICIP'] = df_clean['ID_MUNICIP'].astype('category')
df_clean['ID_REGIONA'] = df_clean['ID_REGIONA'].astype('category')
df_clean['ID_UNIDADE'] = df_clean['ID_UNIDADE'].astype('category')
df_clean['ID_MN_RESI'] = df_clean['ID_MN_RESI'].astype('category')
df_clean['ID_RG_RESI'] = df_clean['ID_RG_RESI'].astype('category')
df_clean['ID_PAIS']    = df_clean['ID_PAIS'].astype('category')
df_clean['CS_SEXO']    = df_clean['CS_SEXO'].astype('category')

df_clean['SEM_PRI']    = df_clean['SEM_PRI'].astype('float')
df_clean['ID_MN_RESI'] = df_clean['ID_MN_RESI'].astype('float')

In [ ]:
df_clean.dtypes

In [ ]:
muni_list = df_clean['MUNICIPIO'].unique().tolist()

In [ ]:
regiao_list = df_clean['REGIAO'].unique().tolist()

In [ ]:
df_clean.groupby(['REGIAO'])['RESUL_SORO'].sum()

In [ ]:
df_clean.groupby(['MUNICIPIO'])['RESUL_SORO'].sum()

## 4.0 Data Transform

### 4.1 Save Clean Data Set

In [ ]:
link = '../dados/df_clean.csv'
df_clean.to_csv(link,sep=',',index=False)

## 5.0 Exploratory Data Analysis

In [ ]:
df_clean.describe()

In [ ]:
df_numeric = df_clean.select_dtypes(include='number')

In [ ]:
plt.figure(figsize=(20,12))
sns.heatmap(df_numeric.corr(), annot=True,cmap='BrBG')
plt.show()
plt.close()

In [ ]:
corr_matrix = df_numeric.corr()
corr_matrix

In [ ]:
colunas = df_numeric.select_dtypes(include='number').columns
ncolunas = len(colunas)

for i in range(0,ncolunas):
 plt.figure(figsize=(20,20))
 sns.displot(x=colunas[i],data=df_numeric)
 plt.show()
 plt.close()

## 6.0 Data Modeling

In [ ]:
# n_components         = 10
# pca                  = PCA(n_components=n_components)
# df_clean_reduzido   = pca.fit_transform(df_clean)
# df_clean_reduzido   = pd.DataFrame(df_clean_reduzido)

## 7.0 Data Preparing

## 8.0 ML Models

### 8.1 Traning

### 8.2 Validation

### 8.3 Measuring

### 8.4 Hyperparamerization

## 9.0 ML Models and Data Visualization